In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

/Users/swithana/git/jupyter/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
columns = ["age", "workclass", "fnlwgt", "education", "education-num", "martial-status",
        "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss",
        "hours-per-week", "native-country", "income"]
data = pd.read_csv('/Users/swithana/git/icicle_model_card/examples/notebooks/data/adult/adult.data', names=columns)

In [3]:
data.head()

,age,workclass,fnlwgt,education,education-num,martial-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


### Pre-processing

In [4]:
label_encoder = LabelEncoder()
data['income'] = label_encoder.fit_transform(data['income'])

# Convert categorical variables to one-hot encoding
data = pd.get_dummies(data, drop_first=True, dtype=float)

data.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,income,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,...,native-country_ Portugal,native-country_ Puerto-Rico,native-country_ Scotland,native-country_ South,native-country_ Taiwan,native-country_ Thailand,native-country_ Trinadad&Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia
0,39,77516,13,2174,0,40,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,50,83311,13,0,0,13,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,38,215646,9,0,0,40,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,53,234721,7,0,0,40,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,28,338409,13,0,0,40,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
X = data.drop('income', axis=1).values
y = data['income'].values

print(data.columns.tolist())

['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week', 'income', 'workclass_ Federal-gov', 'workclass_ Local-gov', 'workclass_ Never-worked', 'workclass_ Private', 'workclass_ Self-emp-inc', 'workclass_ Self-emp-not-inc', 'workclass_ State-gov', 'workclass_ Without-pay', 'education_ 11th', 'education_ 12th', 'education_ 1st-4th', 'education_ 5th-6th', 'education_ 7th-8th', 'education_ 9th', 'education_ Assoc-acdm', 'education_ Assoc-voc', 'education_ Bachelors', 'education_ Doctorate', 'education_ HS-grad', 'education_ Masters', 'education_ Preschool', 'education_ Prof-school', 'education_ Some-college', 'martial-status_ Married-AF-spouse', 'martial-status_ Married-civ-spouse', 'martial-status_ Married-spouse-absent', 'martial-status_ Never-married', 'martial-status_ Separated', 'martial-status_ Widowed', 'occupation_ Adm-clerical', 'occupation_ Armed-Forces', 'occupation_ Craft-repair', 'occupation_ Exec-managerial', 'occupation_ Farming-fishing', 'occupa

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Model Training

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.1, callbacks=[early_stopping])

Epoch 1/100
702/733 [===========================>..] - ETA: 0s - loss: 235.1077 - accuracy: 0.6767

2023-11-13 14:17:00.689808: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at resource_variable_ops.cc:597 : INVALID_ARGUMENT: Cannot update variable with shape [] using a Tensor with shape [0], shapes must be equal.


InvalidArgumentError: Graph execution error:

Detected at node AssignAddVariableOp defined at (most recent call last):
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/runpy.py", line 197, in _run_module_as_main

  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/runpy.py", line 87, in _run_code

  File "/Users/swithana/git/jupyter/venv/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/Users/swithana/git/jupyter/venv/lib/python3.9/site-packages/traitlets/config/application.py", line 1053, in launch_instance

  File "/Users/swithana/git/jupyter/venv/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 736, in start

  File "/Users/swithana/git/jupyter/venv/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 195, in start

  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python

In [9]:
loss, accuracy = model.evaluate(X_test, y_test)

204/204 [==============================] - 0s 529us/step - loss: 36.8450 - accuracy: 0.2409


In [10]:
y_pred = model.predict(X_test)

204/204 [==============================] - 0s 400us/step


## Model Card Generation

In [11]:
from icicle_model_card.icicle_model_card import ModelCard, AIModel, BiasAnalysis, ExplainabilityAnalysis, validate_mc, Metric, save_mc
import json

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [12]:
mc = ModelCard(
            name="UCI Adult Data Analysis",
            version="0.1",
            short_description="UCI Adult Data analysis using Tensorflow",
            full_description="Using a tensorflow trained neural network to analyse fairness and explainability in the UCI Adult Dataset",
            keywords="uci adult, tensorflow, explainability, fairness, fairlearn, shap",
            author="Sachith Withana"
        )

mc.input_data = 'https://archive.ics.uci.edu/dataset/2/adult'
mc.output_data = 'https://github.iu.edu/swithana/mcwork/tensorflow/adult_model.pkl'

In [13]:
model_metrics = []
model_metrics.append(Metric("Test loss", loss))

ai_model = AIModel(
            name="UCI Adult tensorflow model",
            version="0.1",
            description="Census classification problem using Tensorflow Neural Network",
            owner="Sachith Withana",
            location="https://github.iu.edu/swithana/mcwork/tensorflow/adult_model.pkl",
            license="BSD-3 Clause",
            framework="tensorflow",
            model_type="dnn",
            test_accuracy=accuracy,
            model_structure = json.dumps(json.loads(model.to_json()), indent=2)
        )
ai_model.metrics = model_metrics
mc.ai_model = ai_model

In [14]:
y_pred = model.predict(X_test)
y_pred = (y_pred >= 0.5).flatten()
mc.populate_bias(X_test, y_test, y_pred, "gender", X_test[:, 58], model)

204/204 [==============================] - 0s 395us/step


DataFrame.applymap has been deprecated. Use DataFrame.map instead.
DataFrame.applymap has been deprecated. Use DataFrame.map instead.
DataFrame.applymap has been deprecated. Use DataFrame.map instead.
DataFrame.applymap has been deprecated. Use DataFrame.map instead.
DataFrame.applymap has been deprecated. Use DataFrame.map instead.
DataFrame.applymap has been deprecated. Use DataFrame.map instead.


In [20]:
x_columns = data.columns.tolist()
x_columns.remove('income')

mc.populate_xai(X_test, x_columns, model, 10)

PermutationExplainer explainer: 6514it [02:51, 35.67it/s]                                                                                                                                                       


In [21]:
print(mc.bias_analysis)

{'demographic_parity_diff': 0.00045589240939136744, 'equal_odds_difference': 0.001494768310911776, 'true_positives': 1569, 'true_negatives': 0, 'false_positives': 4942, 'false_negatives': 2, 'false_positive_rate': 1.0, 'false_negative_rate': 0.001273074474856779, 'true_positive_rate': 0.9987269255251432, 'true_negative_rate': 0.0}


In [22]:
print(mc.xai_analysis)

{'fnlwgt': 0.0004911673639634201, 'capital-gain': 0.00044787622968457674, 'age': 6.70798050686867e-05, 'hours-per-week': 6.305050295144579e-05, 'martial-status_ Never-married': 2.1357842281867512e-05, 'relationship_ Own-child': 1.6673573169755223e-05, 'relationship_ Not-in-family': 1.3442405517723064e-05, 'occupation_ Other-service': 1.2595947715921443e-05, 'education_ HS-grad': 8.969494734227908e-06, 'relationship_ Other-relative': 8.756830159453904e-06}


In [23]:
print(mc)

{
    "name": "UCI Adult Data Analysis",
    "version": "0.1",
    "short_description": "UCI Adult Data analysis using Tensorflow",
    "full_description": "Using a tensorflow trained neural network to analyse fairness and explainability in the UCI Adult Dataset",
    "keywords": "uci adult, tensorflow, explainability, fairness, fairlearn, shap",
    "author": "Sachith Withana",
    "input_data": "https://archive.ics.uci.edu/dataset/2/adult",
    "output_data": "https://github.iu.edu/swithana/mcwork/tensorflow/adult_model.pkl",
    "ai_model": {
        "name": "UCI Adult tensorflow model",
        "version": "0.1",
        "description": "Census classification problem using Tensorflow Neural Network",
        "owner": "Sachith Withana",
        "location": "https://github.iu.edu/swithana/mcwork/tensorflow/adult_model.pkl",
        "license": "BSD-3 Clause",
        "framework": "tensorflow",
        "model_type": "dnn",
        "test_accuracy": 0.24090281128883362,
        "model_stru

In [17]:
save_mc(mc, "/Users/swithana/git/icicle_model_card/examples/model_cards/tesorflow_adult_nn_MC.json")